In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def matriz_esparsa(imp_fert, quanti_col):
    '''Transforma matriz densa em esparsa.
    Exige que as colunas categoricas entejam como tipo "category"'''
    # salva datas de início e fim
    ini=imp_fert.date.min()
    fim=imp_fert.date.max()
    # Cria conjunto de anos
    anos_df=pd.DataFrame({'CO_ANO':np.arange(ini.year,fim.year+1)}).assign(key=0)
    
    # Funde anos e categorias (inclusive mês) criando nova tabela ms
    # Note que aqui conjunto de carac que não ocorra em determinado mês não é criado com zero quant. para dar velocidade de processamento
    imp_fert_ms=anos_df.merge(\
        imp_fert[imp_fert.select_dtypes('category').columns.tolist()].assign(key=0).drop_duplicates(),how='outer'
    ).drop(columns='key')
        
    # liminta matriz a datas reais existentes
    imp_fert_ms['date']=pd.to_datetime(imp_fert_ms.CO_ANO.astype(str)+'/'+imp_fert_ms.CO_MES.astype(str)+'/1')
    imp_fert_ms=imp_fert_ms[(imp_fert_ms.date>=ini)&(imp_fert_ms.date<=fim)]
    # funde ms e imp_fert
    imp_fert_out=imp_fert_ms.merge(imp_fert,how='outer')
    imp_fert_out[quanti_col]=imp_fert_out[[quanti_col]].fillna(0)

    # Diferencia ocorrências reais de não ocorrência
    imp_fert_out=imp_fert_out.assign(oc=0)
    imp_fert_out.loc[imp_fert_out.KG_LIQUIDO>0,'oc']=1

    return imp_fert_out